Identificação de Fake News com Natural Language Processing (NLP), utilizando o "nltk". 

Para esse exemplo só será utilizado o título da notícia.

In [1]:
# !pip install textblob
# !python -m textblob.download_corpora

In [2]:
import nltk
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
#nltk.download('stopwords')

## Importação e Tratamento dos dados

In [3]:
data = pd.read_csv("train.csv")

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
id        20800 non-null int64
title     20242 non-null object
author    18843 non-null object
text      20761 non-null object
label     20800 non-null int64
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
data = data.drop(['id', 'author', 'text'], axis=1)

In [7]:
data.dropna(inplace = True)

In [8]:
data.columns

Index(['title', 'label'], dtype='object')

In [9]:
data['label'] = data['label'].apply(lambda x: 'true' if x == 0 else 'fake news')
data.label

0        fake news
1             true
2        fake news
3        fake news
4        fake news
           ...    
20795         true
20796         true
20797         true
20798    fake news
20799    fake news
Name: label, Length: 20242, dtype: object

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20242 entries, 0 to 20799
Data columns (total 2 columns):
title    20242 non-null object
label    20242 non-null object
dtypes: object(2)
memory usage: 474.4+ KB


Função para remoção de StopWords:

In [11]:
def remove_stop_words(instance):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = [w for w in instance.split() if not w in stopwords]
    return (" ".join(words))

Função para lemmatization (disponível somente para o idioma inglês):

In [12]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatization(instance):
    words = []
    for w in instance.split():
        words.append(wordnet_lemmatizer.lemmatize(w))
    return (" ".join(words))

Função para remoção de caracteres indesejados:

In [13]:
def data_cleaning(instance):
    instance = re.sub(r"http\S+", "", instance).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instance)

In [14]:
X = data['title']
y = data['label']

In [15]:
X = X.apply(remove_stop_words)
X = X.apply(data_cleaning)
X = X.apply(lemmatization)

## Treinamento

#### Utilizando Pipelines com validação cruzada:

In [16]:
pipeline_mnb = Pipeline([('counts', CountVectorizer()),
                        ('classifier', MultinomialNB())])
pipeline_svm = Pipeline([('counts', CountVectorizer()),
                        ('classifier', svm.LinearSVC())])

Resultado MultinomialNB:

In [17]:
result_mnb = cross_val_predict(pipeline_mnb, X, y, cv=10)

In [18]:
metrics.accuracy_score(y,result_mnb)

0.9064321707341172

In [19]:
metrics.classification_report(y, result_mnb)

'              precision    recall  f1-score   support\n\n   fake news       0.94      0.87      0.90      9855\n        true       0.88      0.94      0.91     10387\n\n    accuracy                           0.91     20242\n   macro avg       0.91      0.91      0.91     20242\nweighted avg       0.91      0.91      0.91     20242\n'

In [20]:
metrics.confusion_matrix(y, result_mnb)

array([[8543, 1312],
       [ 582, 9805]], dtype=int64)

Resultados SVM Linear:

In [21]:
result_svm = cross_val_predict(pipeline_svm, X, y, cv=10)

In [22]:
metrics.accuracy_score(y,result_svm)

0.9362217172216184

In [23]:
metrics.classification_report(y, result_svm)

'              precision    recall  f1-score   support\n\n   fake news       0.92      0.95      0.94      9855\n        true       0.95      0.92      0.94     10387\n\n    accuracy                           0.94     20242\n   macro avg       0.94      0.94      0.94     20242\nweighted avg       0.94      0.94      0.94     20242\n'

In [24]:
metrics.confusion_matrix(y, result_svm)

array([[9380,  475],
       [ 816, 9571]], dtype=int64)